In [ ]:
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC

import warnings
# suppress some MDAnalysis warnings about PSF files
#warnings.filterwarnings('ignore')

from matplotlib import pyplot as plt


: 

In [ ]:
scratchdir = '/home/jvilla/scratch'  # lavandula
scratchdir = '/home/jordivilla/scratchdir'  # laptop

: 

In [ ]:
# some information
u = mda.Universe(scratchdir+'/tre5.parm7',scratchdir+'/mdtre5.nc')
print(u)
print(hasattr(u, 'trajectory'))
print(len(u.trajectory))
print(u.residues)
print(u.atoms)
print(u.select_atoms('resname UNL').n_atoms)
print(u.select_atoms('resid 1').n_atoms)
print(u.atoms[:45].names)

: 

visualize trajectory

In [ ]:
import nglview as nv
view = nv.show_mdanalysis(u.select_atoms('resid 1-5'))
view

: 

In [ ]:
tre = u.select_atoms('resid 1-5')
print(tre.names)
print(tre.center_of_mass())

: 

In [ ]:
for ts in u.trajectory[:20]:
    time = u.trajectory.time
    rgyr = u.atoms.radius_of_gyration()
    print("Frame: {:3d}, Time: {:5.1f} ps, Rgyr: {:.4f} A".format(ts.frame, time, rgyr))

: 

collect data in a list first

In [ ]:
rgyr = []
time = []

for ts in u.trajectory:
    time.append(u.trajectory.time)
    rgyr.append(tre.radius_of_gyration())

: 

now transform intoa  pandas dataframe and plot

In [ ]:
import pandas as pd
rgyr_df = pd.DataFrame(rgyr, columns=['Radius of gyration (A)'], index=time)
rgyr_df.index.name = 'Time (ps)'

rgyr_df.head()

: 

In [ ]:
%matplotlib inline

rgyr_df.plot(title='Radius of gyration')

: 

test the change in RMSD from the first to the last snapshot in the trajectory

In [ ]:
from MDAnalysis.analysis import rms


u.trajectory[0] # first frame
first = tre.positions


u.trajectory[-1] #last frame
last = tre.positions

print(first,last)

rms.rmsd(first, last)

: 

now run a complete RMSD plot

The results are stored in the `.rmsd` attribute. This is an array with the shape `(n_frames, 2 + n_selections)`.

In [ ]:
u.trajectory[0] # set to first frame
rmsd_analysis = rms.RMSD(u, select='resid 1-5', groupselections=['name C', 'name O5'])
rmsd_analysis.run()
print(rmsd_analysis.rmsd.shape)

rmsd_df = pd.DataFrame(rmsd_analysis.rmsd[:, 2:],
                       columns=['Trehalose', 'C', 'O5'],
                       index=rmsd_analysis.rmsd[:, 1])
rmsd_df.index.name = 'Time (ps)'
rmsd_df.head()
rmsd_df.plot(title='RMSD')



: 